In [1]:
import pandas as pd
import numpy as np
import time
import os

## Configuration Variables

In [2]:
CONFIG = 'mrl/' # ['mrl/', 'mrl_e/', 'ff/']
NESTING = CONFIG in ['mrl/', 'mrl_e/']
ROOT_DIR = "../inference/"
DATASET = '1K' # ['1K', '4K', 'V2']
SEARCH_INDEX = 'exactl2' # ['exactl2', 'hnsw_8', 'hnsw_32']
EVAL_CONFIG = 'reranking' # ['vanilla', 'reranking', 'funnel']

'''
nesting_list is used in two ways depending on the config:
1. vanilla: nesting_list = scales at which we retrieve representations for all images
2. reranking: nesting_list = scales at which we rerank representations for all images
3. funnel: unused
'''
if EVAL_CONFIG in ['vanilla', 'reranking']:
    nesting_list = [8, 16, 32, 64, 128, 256, 512, 1024, 2048]
elif EVAL_CONFIG == 'funnel':
    # funnel retrieval
    retrieval_dim = nesting_list = [8]  # for funnel, we evaluate a single config at a time
    # rerank_dim: scale at which neighbors will be re-ordered based on L2 distance
    rerank_dim = [16, 32, 64, 128, 2048]
    # shortlist length which as 1-1 correspondence with rerank_dim
    funnel_shortlist = [800,400,200,50,10]
    CASCADE_NN_FILE = str(retrieval_dim[0])+"dim-cascade"+str(rerank_dim)+"_"+"shortlist"+ \
        str(funnel_shortlist)+"_"+DATASET+"_"+SEARCH_INDEX+".csv"
else:
    raise Exception("Unsupported Evaluation Config.")

'''
ret_dim is used in two ways depending on the config:
1. vanilla: unused
2. reranking: retrieve representations of size ret_dim and rerank with nesting_list
3. funnel: unused
'''
ret_dim = 8

In [3]:
def compute_mAP_recall_at_k(val_classes, db_classes, neighbors, k):
    """
    Computes the MAP@k (default value of k=R) on neighbors with val set by seeing if nearest neighbor
    is in the same class as the class of the val code. Let m be size of val set, and n in train.

      val:          (m x d) All the truncated vector representations of images in val set
      val_classes:  (m x 1) class index values for each vector in the val set
      db_classes:   (n x 1) class index values for each vector in the train set
      neighbors:    (k x m) indices in train set of top k neighbors for each vector in val set
    """

    """
    ImageNet-1K:
    shape of val is: (50000, dim)
    shape of val_classes is: (50000, 1)
    shape of db_classes is: (1281167, 1)
    shape of neighbors is: (50000, 100))
    """

    APs = list()
    precision, recall, topk = [], [], []
    for i in range(val_classes.shape[0]): # Compute precision for each vector's list of k-nn
        target = val_classes[i]
        indices = neighbors[i, :][:k]    # k neighbor list for ith val vector
        labels = db_classes[indices]
        matches = (labels == target)
    
        # topk
        hits = np.sum(matches)
        if hits>0:
            topk.append(1)
        else:
            topk.append(0)
            
        # true positive counts
        tps = np.cumsum(matches)

        # recall
        recall.append(np.sum(matches)/1300)
        precision.append(np.sum(matches)/k)

        # precision values
        precs = tps.astype(float) / np.arange(1, k + 1, 1)
        APs.append(np.sum(precs[matches.squeeze()]) / k)

    return np.mean(APs), np.mean(precision), np.mean(recall), np.mean(topk)

## Load database, query, and neighbor arrays and compute metrics

In [4]:
shortlist = [10, 25, 50, 100] # compute metrics at different shortlist lengths
print("Evaluating at k =", shortlist)

# Load database and query set for nested models
if NESTING:
    # Database: 1.2M x 1 for imagenet1k
    db_labels = np.load(ROOT_DIR + DATASET + "_train_mrl1_e0_ff2048-y.npy")
    # Query set: 50K x 1 for imagenet1k
    query_labels = np.load(ROOT_DIR + DATASET + "_val_mrl1_e0_ff2048-y.npy")
    
for dim in nesting_list:
    start = time.time()
    # Load database and query set for fixed feature models
    if not NESTING:
        db_labels = np.load(ROOT_DIR + DATASET + "_train_mrl0_e0_ff" + str(dim) + "-y.npy")
        query_labels = np.load(ROOT_DIR + DATASET + "_val_mrl0_e0_ff" + str(dim) + "-y.npy")

    # Load neighbors array and compute metrics
    if EVAL_CONFIG == 'reranking':
        print("\nRet Dim: ", ret_dim)
        print("Rerank dim: ", dim)
        neighbors_path = ROOT_DIR + "neighbors/reranked/" + CONFIG + str(ret_dim) + "dim-reranked" \
                    + str(dim) + "_200shortlist_" + DATASET + "_" + SEARCH_INDEX + ".csv"
    elif EVAL_CONFIG == 'vanilla':
        print("\nRet Dim: ", dim)
        neighbors_path = ROOT_DIR + "neighbors/" + CONFIG + SEARCH_INDEX + "_" + str(dim) \
                    + "dim_2048shortlist_" + DATASET + ".csv"
    elif EVAL_CONFIG == 'funnel':
        neighbors_path = ROOT_DIR +"neighbors/funnel_retrieval/" + CONFIG \
                    + CASCADE_NN_FILE
        # remove shortlist elements longer than final funnel dimension
        shortlist = [i for i in shortlist if i <= funnel_shortlist[-1]] 
        print("Updated funnel shortlist k =", shortlist)
    else:
        raise Exception("Unsupported Evaluation Config.")
        
    if not os.path.exists(neighbors_path):
        print(neighbors_path.split("/")[-1] + " not found")
        continue
    neighbors = pd.read_csv(neighbors_path, header=None).to_numpy()
    
    top1 = db_labels[neighbors[:, 0]]
    print("Top1= ", np.sum(top1 == query_labels) / query_labels.shape[0])
    for k in shortlist:
        mAP, precision, recall, topk = compute_mAP_recall_at_k(query_labels, db_labels, neighbors, k)
        print("mAP@%d = %f"%(k, mAP))
        print("precision@%d = %f"%(k, precision))
        print("recall@%d = %f"%(k, recall))
        print("top%d = %f"%(k, topk))
    end = time.time()
    print("Eval time for %d = %0.3f sec\n" %(dim, (end - start)))

Evaluating at k = [10, 25, 50, 100]

Ret Dim:  8
Rerank dim:  8
8dim-reranked8_200shortlist_1K_exactl2.csv not found

Ret Dim:  8
Rerank dim:  16
Top1=  0.6116
mAP@10 = 0.550084
precision@10 = 0.607362
recall@10 = 0.004672
top10 = 0.818980
mAP@25 = 0.533909
precision@25 = 0.602131
recall@25 = 0.011579
top25 = 0.865140
mAP@50 = 0.523875
precision@50 = 0.596741
recall@50 = 0.022952
top50 = 0.892500
mAP@100 = 0.513867
precision@100 = 0.588955
recall@100 = 0.045304
top100 = 0.912460
Eval time for 16 = 7.929 sec


Ret Dim:  8
Rerank dim:  32
8dim-reranked32_200shortlist_1K_exactl2.csv not found

Ret Dim:  8
Rerank dim:  64
8dim-reranked64_200shortlist_1K_exactl2.csv not found

Ret Dim:  8
Rerank dim:  128
8dim-reranked128_200shortlist_1K_exactl2.csv not found

Ret Dim:  8
Rerank dim:  256
8dim-reranked256_200shortlist_1K_exactl2.csv not found

Ret Dim:  8
Rerank dim:  512
8dim-reranked512_200shortlist_1K_exactl2.csv not found

Ret Dim:  8
Rerank dim:  1024
8dim-reranked1024_200shortlist_1K_